# Liste Simplement Chainée

## Principe

Les listes chainées se libèrent de la contrainte de stocker les éléments dans des emplacements contigus en mémoire.

#### Bénéfice:

Aucune déplacement d'élément n'est nécessaire pour 

* l'insertion 
* la suppression 

#### Coûts:

* stocker explicitement l'emplacement de l'élément suivant
* avancer de $k$ positions = passer $k$ fois à l'élément suivant
* utilisation moins efficace de la mémoire cache

Les éléments de la liste sont stockés indivuellement dans les maillons de la chaine. Chaque maillon stocke

* l'élément
* un pointeur vers le maillon suivant

In [1]:
class Maillon:
    def __init__(self,valeur):
        self.donnee = valeur       
        self.suivant = None 
        
    def __str__(self):
        return "{}".format(self.donnee)

Cette structure suffit à constuire une liste manuellement 

In [2]:
tete = Maillon(12) 
tete.suivant = Maillon(99) 
tete.suivant.suivant = Maillon(37) 

Nous utilisons une fonction annexe pour afficher cette liste

In [3]:
import include.liste_simple as h; 
print(h.to_string(tete)) 

## Parcours

Pour traverser tous les éléments d'une liste, il faut 

* traiter le maillon courant
* passer au maillon suivant
* jusqu'à ce que l'on atteigne un maillon nul

On peut l'écrire récursivement

In [4]:
def traverser(M):
    if M != None:
        print(M, end = " → ")
        traverser(M.suivant)
    else:
        print("⌀")

In [5]:
traverser(tete)

12 → 99 → 37 → ⌀


on peut l'écrire itérativement 

In [6]:
def traverser(M):
    while M != None:
        print(M, end = " → ")
        M = M.suivant
    print("⌀")

In [7]:
traverser(tete)

12 → 99 → 37 → ⌀


L'approche itérative est plus appropriée. Une liste peut contenir plus d'éléments que la profondeur de récursion maximale. Les approches récursives sont donc à proscrire. 

## Opérations en tête

Insérer et supprimer en tête sont des opérations simples.

Pour insérer, on doit

* créer un nouveau maillon qui deviendra la tête
* faire de l'ancienne tête le maillon suivant la nouvelle tête

In [8]:
def inserer_en_tete(ancienne_tete, valeur):
    nouvelle_tete = Maillon(valeur)
    nouvelle_tete.suivant = ancienne_tete
    return nouvelle_tete

In [9]:
print(h.to_string(tete)) 
tete = inserer_en_tete(tete,25)
print(h.to_string(tete)) 

12 → 99 → 37 → ⌀
25 → 12 → 99 → 37 → ⌀


Supprimer en tête est encore plus simple. Il suffit de remplacer la tête par l'élément suivant

In [10]:
def supprimer_en_tete(ancienne_tete):
    assert(ancienne_tete)
    nouvelle_tete = ancienne_tete.suivant
    return nouvelle_tete

In [11]:
print(h.to_string(tete)) 
tete = supprimer_en_tete(tete)
print(h.to_string(tete)) 

25 → 12 → 99 → 37 → ⌀
12 → 99 → 37 → ⌀


Attention, dans un langage où il faut détruire explicitement les objets alloués dynamiquement - comme le C++ - il ne faut pas oublier de détruire le maillon supprimé

## Opérations en position quelconque

Insérer ou supprimer après un maillon connu est également efficace. 

Pour insérer un élément derrière le maillon `M`, il faut

* créer un nouveau maillon
* insérer ce maillon entre `M` et `M.suivant`

In [12]:
def inserer_apres(M, valeur):
    assert(M)
    nouveau = Maillon(valeur)
    nouveau.suivant = M.suivant
    M.suivant = nouveau

In [13]:
print(h.to_string(tete)) 
inserer_apres(tete.suivant,42)
print(h.to_string(tete)) 

12 → 99 → 37 → ⌀
12 → 99 → 42 → 37 → ⌀


Pour supprimer l'élément suivant le maillon M, il suivit de faire pointer `M.suivant` vers l'élément suivant celui à supprimer

In [14]:
def supprimer_apres(M):
    assert(M != None and M.suivant != None)
    M.suivant = M.suivant.suivant

In [15]:
print(h.to_string(tete)) 
supprimer_apres(tete)
print(h.to_string(tete)) 

12 → 99 → 42 → 37 → ⌀
12 → 42 → 37 → ⌀


Cette opération nous permet de faire des traitement complexes comme ôter les éléments se répétant en positions successives (comme `std::unique` en C++)

In [16]:
zeros_un_deux = None
for i in range(5):
    for j in range((i+3)//2):
        zeros_un_deux = inserer_en_tete(zeros_un_deux,i%3)

In [17]:
print(h.to_string(zeros_un_deux))

1 → 1 → 1 → 0 → 0 → 0 → 2 → 2 → 1 → 1 → 0 → ⌀


In [18]:
def oter_repetitions(M):
    while M != None and M.suivant != None:
        if M.donnee == M.suivant.donnee:
            supprimer_apres(M)
        else:
            M = M.suivant

In [19]:
oter_repetitions(zeros_un_deux)
print(h.to_string(zeros_un_deux))

1 → 0 → 2 → 1 → 0 → ⌀


## Classe `forward_list`

S'il est possible de se contenter de la classe (ou structure) `Maillon`, il peut être utile de définir une classe `forward_list` qui 

* se souvient de **quel maillon est la tête**. On évite ainsi de devoir écrire `tete = inserer_en_tete(tete,25)`, qui est source d'erreurs si l'on oublie d'affecter le résultat à `tete`



* stocke éventuellement d'autres informations comme la **taille de la liste**. Sinon, calculer la taille d'une liste de $n$ éléments est un opération de complexité $\Theta(n)$


Les noms de la classe et des méthodes singent ceux de la classe `std::forward_list` du C++. Mais attention, celle-ci ne stocke pas la taille et n'a donc pas de méthode `size()`

In [20]:
class forward_list:
    def __init__(self):
        self.__t = None # tête
        self.__n = 0    # taille
        
    def __str__(self): return h.to_string(self.__t)
    
    def empty(self):   return self.__t == None
    
    def size(self):    return self.__n
    
    def push_front(self,valeur):
        self.__t = inserer_en_tete(self.__t,valeur); 
        self.__n += 1

    def pop_front(self):
        self.__t = supprimer_en_tete(self.__t); 
        self.__n -= 1
        
    def insert_after(self,m,valeur):
        inserer_apres(m,valeur); 
        self.__n += 1

    def erase_after(self,m):
        supprimer_apres(m); 
        self.__n -= 1

Exemples d'utilisation

In [21]:
L = forward_list()
for i in range(10):
    L.push_front(i%4)
print(L)

1 → 0 → 3 → 2 → 1 → 0 → 3 → 2 → 1 → 0 → ⌀


In [22]:
L.pop_front()
print(L)

0 → 3 → 2 → 1 → 0 → 3 → 2 → 1 → 0 → ⌀


In [23]:
print(L.size())

9


## Itérer sur une liste 

Pour rendre cette classe réellement utile, il faut pouvoir **boucler sur ses éléments** indépendamment de sa structure interne. 

Pour cela, nous introduisons les fonctions suivantes qui cachent entièrement les notions de maillon, de tête de chaine, ... 

In [24]:
def debut(L):     return L._forward_list__t

def fin(L):       return None

def suivant(m):   return m.suivant

def get_val(m):   return m.donnee

def set_val(m,v): m.donnee = v

*Note:* ` L._forward_list__t` est le nom externe de l'attribut "privé" `__t` en python

Ces fonctions correspondent à peu près en C++ à 

* `std::forward_list<T>::begin()` 


* `std::forward_list<T>::end()`


* `std::forward_list<T>::iterator::operator++()`


* `std::forward_list<T>::iterator::operator*()`


Exemples d'utilisation

In [25]:
def afficher(L):
    m = debut(L)
    while m != fin(L):
        print(get_val(m), end=" ")
        m = suivant(m)
        
afficher(L)

0 3 2 1 0 3 2 1 0 

In [26]:
def incrementer_tout(L):
    m = debut(L)
    while m != fin(L):
        set_val(m, get_val(m)+1)
        m = suivant(m)

print(L); incrementer_tout(L); print(L)

0 → 3 → 2 → 1 → 0 → 3 → 2 → 1 → 0 → ⌀
1 → 4 → 3 → 2 → 1 → 4 → 3 → 2 → 1 → ⌀


In [27]:
def inserer_zero_apres_un(L):
    m = debut(L)
    while m != fin(L):
        if get_val(m) == 1:
            L.insert_after(m,0)
        m = suivant(m)

print(L); inserer_zero_apres_un(L); print(L)

1 → 4 → 3 → 2 → 1 → 4 → 3 → 2 → 1 → ⌀
1 → 0 → 4 → 3 → 2 → 1 → 0 → 4 → 3 → 2 → 1 → 0 → ⌀


In [28]:
def supprimer_valeurs(L,val):
    m = debut(L)
    while m != fin(L):
        if suivant(m) and get_val(suivant(m)) == val:
            L.erase_after(m)
        else: m = suivant(m)
            
print(L); supprimer_valeurs(L,0); print(L)

1 → 0 → 4 → 3 → 2 → 1 → 0 → 4 → 3 → 2 → 1 → 0 → ⌀
1 → 4 → 3 → 2 → 1 → 4 → 3 → 2 → 1 → ⌀


Mais attention, cette fonction n'est pas correctement écrite. Essayons de supprimer la valeur 1

In [29]:
print(L); supprimer_valeurs(L,1); print(L)

1 → 4 → 3 → 2 → 1 → 4 → 3 → 2 → 1 → ⌀
1 → 4 → 3 → 2 → 4 → 3 → 2 → ⌀


La valeur en tête n'est pas supprimée. 

Il est impossible avec le faire avec `erase_after`, puisqu'elle est en tête et pas après un autre maillon. 

Comment faire? 

* ré-écrire la fonction `supprimer_valeurs` ?


* ré-écrire la classe `forward_list` ! 

## `forward_list` améliorée

Le problème étant que l'on avait pas de maillon avant la tête,

la solution est simple: plaçons un maillon **avant la tête**

Ce maillon ne stocke pas d'élément, il sert uniquement à stocker le maillon de tête dans son attribut `suivant`. 

Le plus proprement possible, il a son propre type

In [30]:
class MaillonVide:
    def __init__(self):
        self.suivant = None 

Réécrire le code de la classe est immédiat, et légèrement simplifié puisque `push_front` peut maintenant appeler `insert_after`

In [31]:
class forward_list:
    def __init__(self):
        self.__av = MaillonVide() # avant tête
        self.__n = 0              # taille  
        
    def __str__(self): return h.to_string(self.__av.suivant)

    def empty(self):   return self.__n == 0
    
    def size(self):    return self.__n
    
    def insert_after(self,m,val):
        inserer_apres(m,val); 
        self.__n += 1
        
    def push_front(self,val):
        self.insert_after(self.__av,val)
        
    def erase_after(self,m):
        supprimer_apres(m); 
        self.__n -= 1
        
    def pop_front(self):
        self.erase_after(self.__av)

Pour itérer, on introduit une nouvelle fonction `avant_debut` et on redéfinit `debut`. Les autres sont inchangées

In [32]:
def avant_debut(L): return L._forward_list__av

def debut(L):       return suivant(avant_debut(L))

def fin(L):         return None

def suivant(m):     return m.suivant

def get_val(m):     return m.donnee

def set_val(m,v):   m.donnee = v

*Note:* la fonction `avant_debut` correspond à `std::forward_list<T>::before_begin()` en C++

La fonction 'supprimer_valeur' se réécrit en itérant depuis `avant_debut(L)`

In [33]:
def supprimer_valeurs(L,val):
    courant = avant_debut(L)
    precedent = courant
    while courant != fin(L):
        courant = suivant(courant)
        if courant and get_val(courant) == val:
            L.erase_after(precedent)
        else:
            precedent = courant

In [34]:
L = forward_list()
for i in range(10): L.push_front(i%4)
print(L)
supprimer_valeurs(L,0)
print(L)

1 → 0 → 3 → 2 → 1 → 0 → 3 → 2 → 1 → 0 → ⌀
1 → 3 → 2 → 1 → 3 → 2 → 1 → ⌀


## Copie de liste

La copie complète d'un liste peut paraitre simple, mais une approche naive ne fonctionne pas

In [35]:
def copie_liste_naive(L):
    C = forward_list()
    courant = debut(L);
    while courant != fin(L):
        C.push_front(get_val(courant))
        courant = suivant(courant)
    return C

In [36]:
L1 = forward_list()
for i in range(10): L1.push_front(i%4)
print(L1); L2 = copie_liste_naive(L1); print(L2)

1 → 0 → 3 → 2 → 1 → 0 → 3 → 2 → 1 → 0 → ⌀
0 → 1 → 2 → 3 → 0 → 1 → 2 → 3 → 0 → 1 → ⌀


Pour copier sans inverser, il faut insérer en queue et pas en tête. 

Cette opération n'est pas directement disponible, mais elle le devient si l'on se souvient du dernier élément de la liste en construction. 

In [37]:
def copie_liste(L):
    C = forward_list()
    courant = debut(L);
    dernier_element_C = avant_debut(C)
    while courant != fin(L):
        C.insert_after(dernier_element_C,get_val(courant))
        dernier_element_C = suivant(dernier_element_C)
        courant = suivant(courant)
    return C

In [38]:
print(L1); L2 = copie_liste(L1); print(L2)

1 → 0 → 3 → 2 → 1 → 0 → 3 → 2 → 1 → 0 → ⌀
1 → 0 → 3 → 2 → 1 → 0 → 3 → 2 → 1 → 0 → ⌀


## Conclusions

Une liste simplement chainée utilise comme maillons des structures stockant individuellement

* un élément
* un lien vers l'élément suivant

Les opérations efficaces en $\Theta(1)$ sont

* insertion et suppression en tête
* insertion et suppression après une position connue

Il n'y a ni pas d'accès indexé ou d'opération en queue. 

La mise en oeuvre de l'itération et l'utilisation d'un noeud vide en tête simplifient l'utilisation de la structure. 

<table style="width: 100%; border: 0px">
<tr style="background-color:white; border:0px">
<td style="width: 120px; border: 0px">
    <img src="https://heig-vd.ch/ResourcePackages/WhiteFox/assets/images/logo-heig-vd.svg" height=200px align=left >
    </td>
    <td style="vertical-align: middle; border: 0px" height=200px>
    <p style="text-align: left">
        <a href="https://ocuisenaire.github.io/ASD1-notebooks/">ASD1 Notebooks on GitHub.io</a>
 </p>        
<p style="text-align: left">
© Olivier Cuisenaire, 2018 </p>
</td>
</tr>
</table>